In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-02 22:24:17--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   106MB/s    in 0.2s    

2025-06-02 22:24:17 (106 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Step 1: Get user and book rating counts
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Step 2: Filter users with >= 200 ratings and books with >= 100 ratings
users_to_keep = user_counts[user_counts >= 200].index
books_to_keep = book_counts[book_counts >= 100].index

# Step 3: Filter the ratings dataframe
df_ratings_filtered = df_ratings[
    df_ratings['user'].isin(users_to_keep) &
    df_ratings['isbn'].isin(books_to_keep)
]

# Step 4: Merge with books to get titles
df_final = df_ratings_filtered.merge(df_books, on='isbn')

# Step 5: Remove books with ratings of 0 (implicit feedback)
df_final = df_final[df_final['rating'] != 0]

# Step 6: Create user-book matrix
user_book_matrix = df_final.pivot_table(
    index='user',
    columns='title',
    values='rating'
).fillna(0)

# Step 7: Transpose to get book-user matrix
book_user_matrix = user_book_matrix.T

# Step 8: Convert to sparse matrix
book_user_sparse = csr_matrix(book_user_matrix.values)

# Step 9: Fit KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(book_user_sparse)

def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    # Get book index
    book_idx = list(book_user_matrix.index).index(book)

    # Find neighbors
    distances, indices = model_knn.kneighbors(
        book_user_matrix.iloc[book_idx].values.reshape(1, -1),
        n_neighbors=6
    )

    # Create recommendations (skip the first one as it's the book itself)
    recommendations = []
    for i in range(1, len(distances.flatten())):
        rec_book = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommendations.append([rec_book, distance])

    # Sort by distance in DESCENDING order (highest distance first)
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return [book, recommendations]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
